In [ ]:
%load_ext autoreload
%autoreload 2
from gui import SpikeImportGUI
from importers import SpikeImporter

# present the Spike GUI and let the user make his choices
spike_gui = SpikeImportGUI()

importer = SpikeImporter(filepath = spike_gui.filepath, time_channel = spike_gui.time_channel, signal_channel = spike_gui.signal_channel)
    
# retrieve the events from the importer 
el_stimuli = importer.get_electrical_stimuli(regular_stimulus_channel = spike_gui.stimulus_channels["regular_electrical"])

if spike_gui.load_mech.get() == True:
    mech_stimuli = importer.get_mechanical_stimuli(force_channel = spike_gui.stimulus_channels["force"], threshold = spike_gui.force_threshold, max_gap_time = 0.005)
else:
    mech_stimuli = []
    
if spike_gui.load_ex_el.get() == True:
    ex_stimuli = importer.get_extra_stimuli(extra_stimulus_channel = spike_gui.stimulus_channels["extra_electrical"], regular_el_stimuli = el_stimuli)
else:
    ex_stimuli = []
    
actpots = importer.get_action_potentials(max_gap_time = spike_gui.max_ap_gap, ap_marker_channels = spike_gui.ap_channels, \
                                       el_stimuli = el_stimuli, mech_stimuli = mech_stimuli)

raw_signal = importer.get_raw_signal_split_by_stimuli(el_stimuli = el_stimuli, verbose = False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
'''
Here, we extract some features from the APs:
- postpulse distance (aka latency)
- normalized signal energy
'''
%autoreload 2
from tqdm import tqdm
from feature_extraction import calc_dist_to_prev_reg_el_stimulus, calc_normalized_energy

print("Extracting features from every AP")

# all_el_stimuli = el_stimuli.copy()
# all_el_stimuli.extend(importer.get_electrical_stimuli(regular_stimulus_channel = extra_stimuli_channel))

for ap in tqdm(actpots):
    ap.features["latency"], ap.prev_stimuli["regular"] = calc_dist_to_prev_reg_el_stimulus(ap, el_stimuli)
    ap.features["energy"] = calc_normalized_energy(ap)

100%|█████████████████████████████████████████████████████████████████████████████| 513/513 [00:00<00:00, 19007.42it/s]

Extracting features from every AP


In [101]:
''' 
This cell performs the clustering according to
- distance to previous stimulus
- normalized signal energy of the APs
'''
%autoreload 2
from fibre_tracking import DBSCANClustering
from plotting import ClusterPlot2D

clustered_data_df = DBSCANClustering.perform_clustering(actpots, eps = 0.002, min_samples = 10, save_fibre_prediction = True)

cl2dplot = ClusterPlot2D()
cl2dplot.plot(clustered_data_df = clustered_data_df)